In [1]:
# import必要套件
from urllib import parse
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from time import time, sleep
from random import randint
import os
import re

In [2]:
# 傳入URL、設定隨機UserAgent、發出請求並確認狀況
url = 'https://ncode.syosetu.com/n4191cj/'

# 設定隨機UserAgent
ua = UserAgent()
my_headers = {
    'User-Agent':ua.random
}

# 發出get request，確認連線狀況
response = req.get(url, headers = my_headers)
print(f'網站狀態碼: {response.status_code}')
print(f'網站編碼　: {response.encoding}')
print(f'回覆標頭　: {response.headers}')

網站狀態碼: 200
網站編碼　: UTF-8
回覆標頭　: {'Date': 'Sun, 21 Jan 2024 08:06:37 GMT', 'Server': 'Apache', 'Set-Cookie': 'ks2=sfdnsaf344yf; expires=Mon, 20-Jan-2025 08:06:38 GMT; Max-Age=31536000; path=/; domain=syosetu.com; secure', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Connection': 'close', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=UTF-8'}


In [3]:
# 開始爬蟲 - 建立soup物件、取得小說名

# 建立soup物件用lxml parser
soup = bs(response.text, 'lxml')

# 小說標題為只有一個內容的iterable物件，用index 0取出值
novel_title = soup.select('div#novel_contents p.novel_title')[0].get_text()
print(novel_title)

美女と賢者と魔人の剣


c:\Users\John1\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
novel_content = soup.select('div.index_box')

for content in novel_content:
    print(content)

<div class="index_box">
<div class="chapter_title">プロローグ</div>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/1/">000 落下</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2019/08/21 23:58 改稿">（<u>改</u>）</span></dt>
</dl>
<div class="chapter_title">第一部　カリス篇</div>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/2/">001 世界</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2016/01/04 15:52 改稿">（<u>改</u>）</span></dt>
</dl>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/3/">002 決断</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2016/01/04 15:31 改稿">（<u>改</u>）</span></dt>
</dl>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/4/">003 教会 ★</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2016/01/04 15:53 改稿">（<u>改</u>）</span></dt>
</dl>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/5/">004 疑念</a>
</dd>
<dt class="long_update">
2014/11

In [5]:
# 確認小說本體(div.index_box內部構造)
novel_content = soup.select('div.index_box')

regexChapterTitle = r'<div class=\"chapter_title\">([一-龯ぁ-んァ-ン　\w]+)<\/div>'
# index_box內為一個巨大的content串列 (只有一個元素)
# for element in novel_content:
#     print(f"=====\n{element}\n=====")

# print(novel_content)
print(re.search(regexChapterTitle, str(novel_content))[1])

print(f"novel_content元素類型: {type(novel_content)}")
print(f"novel_content內元素數: {len(novel_content)}")
print(f"novel_content內文: \n{novel_content}")

プロローグ
novel_content元素類型: <class 'bs4.element.ResultSet'>
novel_content內元素數: 1
novel_content內文: 
[<div class="index_box">
<div class="chapter_title">プロローグ</div>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/1/">000 落下</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2019/08/21 23:58 改稿">（<u>改</u>）</span></dt>
</dl>
<div class="chapter_title">第一部　カリス篇</div>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/2/">001 世界</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2016/01/04 15:52 改稿">（<u>改</u>）</span></dt>
</dl>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/3/">002 決断</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2016/01/04 15:31 改稿">（<u>改</u>）</span></dt>
</dl>
<dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/4/">003 教会 ★</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2016/01/04 15:53 改稿">（<u>改</u>）</span></dt>
</dl>
<dl class="novel_sublist

In [6]:
# div.index_box內第一個層級有div以及dl 2種元素標籤，設定讓select 2種都算
novel_content = soup.select('div.index_box')
content = novel_content[0].select('div, dl')

for item in content[0:3]:
    print(f"item: {item}")
    print("="*50)

print("\n\n===== 分隔線 =====\n\n")

novel_content2 = soup.select('div.index_box div,dl')
for item in novel_content2[0:3]:
    print(f"item: {item}")
    print("="*50)

item: <div class="chapter_title">プロローグ</div>
item: <dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/1/">000 落下</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2019/08/21 23:58 改稿">（<u>改</u>）</span></dt>
</dl>
item: <div class="chapter_title">第一部　カリス篇</div>


===== 分隔線 =====


item: <div class="chapter_title">プロローグ</div>
item: <dl class="novel_sublist2">
<dd class="subtitle">
<a href="/n4191cj/1/">000 落下</a>
</dd>
<dt class="long_update">
2014/11/09 17:00<span title="2019/08/21 23:58 改稿">（<u>改</u>）</span></dt>
</dl>
item: <div class="chapter_title">第一部　カリス篇</div>


In [7]:
novel_content = soup.select('div.index_box div,dl')
chapterInfo = []
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'

# for item in novel_content:
#     print(item)
#     print("=" * 50)

section = ''
for item in novel_content[0:20]:
    if item['class']==['chapter_title']:
        section = item.get_text()
        continue
    else:
        chapterTitle = item.a.get_text()
        uploadDateTime = re.search(regexDateTime, item.dt.get_text())[0]
        updateDateTime = ''
        if item.find('span') != None:
            updateDateTime = re.search(regexDateTime, item.span['title'])[0]
        chapterLink = f"https://ncode.syosetu.com{item.a['href']}"

    link = [section, chapterTitle, uploadDateTime, updateDateTime, chapterLink]
    chapterInfo.append(link)

# 確認抓到資料(for-each)
for item in chapterInfo:
    print(f"章名:{item[0]}　節名:{item[1]}　上傳時間:{item[2]}　更新時間:{item[3]}　連結:{item[4]}")

章名:プロローグ　節名:000 落下　上傳時間:2014/11/09 17:00　更新時間:2019/08/21 23:58　連結:https://ncode.syosetu.com/n4191cj/1/
章名:第一部　カリス篇　節名:001 世界　上傳時間:2014/11/09 17:00　更新時間:2016/01/04 15:52　連結:https://ncode.syosetu.com/n4191cj/2/
章名:第一部　カリス篇　節名:002 決断　上傳時間:2014/11/09 17:00　更新時間:2016/01/04 15:31　連結:https://ncode.syosetu.com/n4191cj/3/
章名:第一部　カリス篇　節名:003 教会 ★　上傳時間:2014/11/09 17:00　更新時間:2016/01/04 15:53　連結:https://ncode.syosetu.com/n4191cj/4/
章名:第一部　カリス篇　節名:004 疑念　上傳時間:2014/11/09 17:00　更新時間:2016/02/15 14:55　連結:https://ncode.syosetu.com/n4191cj/5/
章名:第一部　カリス篇　節名:005 使徒　上傳時間:2014/11/09 18:00　更新時間:2016/01/04 15:47　連結:https://ncode.syosetu.com/n4191cj/6/
章名:第一部　カリス篇　節名:006 奇襲　上傳時間:2014/11/09 19:00　更新時間:2016/01/04 15:53　連結:https://ncode.syosetu.com/n4191cj/7/
章名:第一部　カリス篇　節名:007 邸宅　上傳時間:2014/11/09 20:00　更新時間:2016/01/04 15:56　連結:https://ncode.syosetu.com/n4191cj/8/
章名:第一部　カリス篇　節名:008 決戦　上傳時間:2014/11/09 21:00　更新時間:2016/01/04 15:59　連結:https://ncode.syosetu.com/n4191cj/9/
章名:第一部　カリス篇　節名:009 覚悟　上傳時間:2014/11/09 22:00　更新時

In [8]:
# 抓出章名以及其中小說節名
novel_content = soup.select('div.index_box')
regexChapterTitle = r'<div class=\"chapter_title\">([一-龯ぁ-んァ-ン　\w]+)<\/div>'
chapterInfoSource = soup.select('div.index_box dl.novel_sublist2')
chapterInfo = []
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'
howMany = 20

# 不限制取出筆數
# for item in chapterInfoSource:
#     sectionSliceIndex = str(novel_content).find(item.a.get_text())
#     sectionString = str(novel_content)[0:sectionSliceIndex]
#     section = re.findall(regexChapterTitle, sectionString)
#     latest_section = section[len(section)-1]
#     chapterTitle = item.a.get_text()
#     uploadDateTime = re.search(regexDateTime, item.dt.get_text())[0]
#     updateDateTime = ''
#     if item.find('span') != None:
#         updateDateTime = re.search(regexDateTime, item.span['title'])[0]
#     chapterLink = f"https://ncode.syosetu.com{item.a['href']}"

#     link = [latest_section, chapterTitle, uploadDateTime, updateDateTime, chapterLink]
#     chapterInfo.append(link)


# 限制取出筆數
for i in range(howMany):
    sectionSliceIndex = str(novel_content).find(chapterInfoSource[i].a.get_text())
    sectionString = str(novel_content)[0:sectionSliceIndex]
    section = re.findall(regexChapterTitle, sectionString)
    latest_section = section[len(section)-1]
    chapterTitle = chapterInfoSource[i].a.get_text()
    uploadDateTime = re.search(regexDateTime, chapterInfoSource[i].dt.get_text())[0]
    updateDateTime = ''
    if chapterInfoSource[i].find('span') != None:
        updateDateTime = re.search(regexDateTime, chapterInfoSource[i].span['title'])[0]
    chapterLink = f"https://ncode.syosetu.com{chapterInfoSource[i].a['href']}"


    link = [latest_section, chapterTitle, uploadDateTime, updateDateTime, chapterLink]
    chapterInfo.append(link)

    
# 確認抓到資料        
# i = 0
# for i in range(50):
#     print(f"{i+1}. {chapterInfo[i]}")


# 確認抓到資料(for-each)
for item in chapterInfo:
    print(f"章名:{item[0]}　節名:{item[1]}　上傳時間:{item[2]}　更新時間:{item[3]}　連結:{item[4]}")



# 一筆一筆抓出每個
# for title in chapter_titles:
#     sectionSliceIndex = str(novel_content).find(title.a.get_text())
#     sectionString = str(novel_content)[0:sectionSliceIndex]
#     section = re.findall(regexChapterTitle, sectionString)
#     latest_section = section[len(section)-1]
#     print(f"{latest_section}　　{title.a.get_text()}")

章名:プロローグ　節名:000 落下　上傳時間:2014/11/09 17:00　更新時間:2019/08/21 23:58　連結:https://ncode.syosetu.com/n4191cj/1/
章名:第一部　カリス篇　節名:001 世界　上傳時間:2014/11/09 17:00　更新時間:2016/01/04 15:52　連結:https://ncode.syosetu.com/n4191cj/2/
章名:第一部　カリス篇　節名:002 決断　上傳時間:2014/11/09 17:00　更新時間:2016/01/04 15:31　連結:https://ncode.syosetu.com/n4191cj/3/
章名:第一部　カリス篇　節名:003 教会 ★　上傳時間:2014/11/09 17:00　更新時間:2016/01/04 15:53　連結:https://ncode.syosetu.com/n4191cj/4/
章名:第一部　カリス篇　節名:004 疑念　上傳時間:2014/11/09 17:00　更新時間:2016/02/15 14:55　連結:https://ncode.syosetu.com/n4191cj/5/
章名:第一部　カリス篇　節名:005 使徒　上傳時間:2014/11/09 18:00　更新時間:2016/01/04 15:47　連結:https://ncode.syosetu.com/n4191cj/6/
章名:第一部　カリス篇　節名:006 奇襲　上傳時間:2014/11/09 19:00　更新時間:2016/01/04 15:53　連結:https://ncode.syosetu.com/n4191cj/7/
章名:第一部　カリス篇　節名:007 邸宅　上傳時間:2014/11/09 20:00　更新時間:2016/01/04 15:56　連結:https://ncode.syosetu.com/n4191cj/8/
章名:第一部　カリス篇　節名:008 決戦　上傳時間:2014/11/09 21:00　更新時間:2016/01/04 15:59　連結:https://ncode.syosetu.com/n4191cj/9/
章名:第一部　カリス篇　節名:009 覚悟　上傳時間:2014/11/09 22:00　更新時

In [9]:
# 抓出章名以及其中小說節名
novel_content = soup.select('div.index_box')
regexChapterTitle = r'<div class=\"chapter_title\">([一-龯ぁ-んァ-ン　\w]+)<\/div>'
chapterInfoSource = soup.select('div.index_box dl.novel_sublist2')
chapterInfoDict = {'latest_section':[], 'chapterTitle':[], 'uploadDateTime':[], 'updateDateTime':[], 'chapterLink':[]}
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'



# 限制取出筆數
for i in range(10):
    sectionSliceIndex = str(novel_content).find(chapterInfoSource[i].a.get_text())
    sectionString = str(novel_content)[0:sectionSliceIndex]
    section = re.findall(regexChapterTitle, sectionString)
    latest_section = section[len(section)-1]
    chapterTitle = chapterInfoSource[i].a.get_text()
    uploadDateTime = re.search(regexDateTime, chapterInfoSource[i].dt.get_text())[0]
    updateDateTime = ''
    if chapterInfoSource[i].find('span') != None:
        updateDateTime = re.search(regexDateTime, chapterInfoSource[i].span['title'])[0]
    chapterLink = f"https://ncode.syosetu.com{chapterInfoSource[i].a['href']}"

    chapterInfoDict['latest_section'].append(latest_section)
    chapterInfoDict['chapterTitle'].append(chapterTitle)
    chapterInfoDict['uploadDateTime'].append(uploadDateTime)
    chapterInfoDict['updateDateTime'].append(updateDateTime)
    chapterInfoDict['chapterLink'].append(chapterLink)
    
# print(chapterInfoDict)

for i in range(10):
    print(chapterInfoDict['latest_section'][i], end='')
    print(chapterInfoDict['chapterTitle'][i], end='')
    print(chapterInfoDict['uploadDateTime'][i], end='')
    print(chapterInfoDict['updateDateTime'][i], end='')
    print(chapterInfoDict['chapterLink'][i])
    print("="*50)


プロローグ000 落下2014/11/09 17:002019/08/21 23:58https://ncode.syosetu.com/n4191cj/1/
第一部　カリス篇001 世界2014/11/09 17:002016/01/04 15:52https://ncode.syosetu.com/n4191cj/2/
第一部　カリス篇002 決断2014/11/09 17:002016/01/04 15:31https://ncode.syosetu.com/n4191cj/3/
第一部　カリス篇003 教会 ★2014/11/09 17:002016/01/04 15:53https://ncode.syosetu.com/n4191cj/4/
第一部　カリス篇004 疑念2014/11/09 17:002016/02/15 14:55https://ncode.syosetu.com/n4191cj/5/
第一部　カリス篇005 使徒2014/11/09 18:002016/01/04 15:47https://ncode.syosetu.com/n4191cj/6/
第一部　カリス篇006 奇襲2014/11/09 19:002016/01/04 15:53https://ncode.syosetu.com/n4191cj/7/
第一部　カリス篇007 邸宅2014/11/09 20:002016/01/04 15:56https://ncode.syosetu.com/n4191cj/8/
第一部　カリス篇008 決戦2014/11/09 21:002016/01/04 15:59https://ncode.syosetu.com/n4191cj/9/
第一部　カリス篇009 覚悟2014/11/09 22:002016/01/04 16:05https://ncode.syosetu.com/n4191cj/10/


In [12]:
folderPath = novel_title
print(folderPath)

美女と賢者と魔人の剣


In [13]:
# 建立章節資料夾
# illegal_c = 不能出現在資料夾名判定用字串

illegal_c = '[@_!#$%^&*()<>?/|}{~:]'


for item in chapterInfo:
    
    # ===== 建立章資料夾 =====
    chapterPath = ''
    for char in item[0]:
        if char in illegal_c:
            truncateIndex = item[0].find(char)
            chapterPath = f"{folderPath}/{item[0][0:truncateIndex]}"
            break
        else:
            chapterPath = f"{folderPath}/{item[0]}"

    if not os.path.exists(chapterPath):
        os.makedirs(chapterPath)
    # ===== 區塊結束 =====
    
    # ===== 建立爬文章本文需要的呼叫、soup物件，以及檔名 =====
    res_chapter = req.get(link[4], headers = my_headers)
    soup_chapter = bs(res_chapter.text, 'lxml')
    chapter_content = soup_chapter.select('div#novel_honbun p')
    chapter_title = item[1]
    
    for char in chapter_title:
        if char in illegal_c:
            chapter_title = chapter_title.replace(char, '')
    # ===== 區塊結束 =====
    
    # ===== 開啟檔案並寫入 =====
    with open(f'{chapterPath}/{chapter_title}.txt', 'w', encoding='utf-8') as novel_file:
        novel_file.write(f"タイトル：{item[1]}\n更新日時：{item[2]}\n最終更新：{item[3]}\n\n\n")
        for text in chapter_content:
            novel_file.write(f"{text.get_text()}\n")

    # 爬連結間空3~7秒避免因太過頻繁被認定是爬蟲
    sleep(randint(3, 7))
    # ===== 區塊結束 =====


c:\Users\John1\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [14]:
# 取得、印出各小說章節連結list
chapter_link = []
for link in soup.select('div.index_box dd.subtitle a'):
    chapter_link.append(f"https://ncode.syosetu.com{link['href']}")
    
for link in chapter_link:
    print(link)

https://ncode.syosetu.com/n4191cj/1/
https://ncode.syosetu.com/n4191cj/2/
https://ncode.syosetu.com/n4191cj/3/
https://ncode.syosetu.com/n4191cj/4/
https://ncode.syosetu.com/n4191cj/5/
https://ncode.syosetu.com/n4191cj/6/
https://ncode.syosetu.com/n4191cj/7/
https://ncode.syosetu.com/n4191cj/8/
https://ncode.syosetu.com/n4191cj/9/
https://ncode.syosetu.com/n4191cj/10/
https://ncode.syosetu.com/n4191cj/11/
https://ncode.syosetu.com/n4191cj/12/
https://ncode.syosetu.com/n4191cj/13/
https://ncode.syosetu.com/n4191cj/14/
https://ncode.syosetu.com/n4191cj/15/
https://ncode.syosetu.com/n4191cj/16/
https://ncode.syosetu.com/n4191cj/17/
https://ncode.syosetu.com/n4191cj/18/
https://ncode.syosetu.com/n4191cj/19/
https://ncode.syosetu.com/n4191cj/20/
https://ncode.syosetu.com/n4191cj/21/
https://ncode.syosetu.com/n4191cj/22/
https://ncode.syosetu.com/n4191cj/23/
https://ncode.syosetu.com/n4191cj/24/
https://ncode.syosetu.com/n4191cj/25/
https://ncode.syosetu.com/n4191cj/26/
https://ncode.syosetu

In [15]:
# 取得各小說章節連結 - 追加章節名、上傳時間、更新時間，存入串列的串列格式
# print(soup.select('div.index_box dl.novel_sublist2')[0])

# # 測試output用
# print(soup.select('div.index_box dl.novel_sublist2')[0].a.get_text())
# print(soup.select('div.index_box dl.novel_sublist2')[0].a['href'])
# print(soup.select('div.index_box dl.novel_sublist2')[0].dt.get_text())
# print(soup.select('div.index_box dl.novel_sublist2')[0].dt.span['title'])

chapterInfoSource = soup.select('div.index_box dl.novel_sublist2')
regexDateTime = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'

chapterInfo = []
for item in chapterInfoSource:
    chapterTitle = item.a.get_text()
    uploadedTime = re.search(regexDateTime, item.dt.get_text())
    # uploadedTime = item.dt.get_text()
    updateTime = ''
    if item.find('span') != None:
        updateTime = re.search(regexDateTime, item.span['title'])[2]
        # updateTime = item.span['title']
    chapterLink = f"https://ncode.syosetu.com{item.a['href']}"
        
    link = [chapterTitle, uploadedTime, updateTime, chapterTitle]
            
    chapterInfo.append(link)

i = 0
for i in range(len(chapterInfo)):
    print(f"{i+1}. {chapterInfo}")

# # repr()確認是否含特殊符號
# print(repr(soup.select('div.index_box dl.novel_sublist2')[0].a.get_text()))

1. [['000 落下', <re.Match object; span=(1, 17), match='2014/11/09 17:00'>, '23:58', '000 落下'], ['001 世界', <re.Match object; span=(1, 17), match='2014/11/09 17:00'>, '15:52', '001 世界'], ['002 決断', <re.Match object; span=(1, 17), match='2014/11/09 17:00'>, '15:31', '002 決断'], ['003 教会 ★', <re.Match object; span=(1, 17), match='2014/11/09 17:00'>, '15:53', '003 教会 ★'], ['004 疑念', <re.Match object; span=(1, 17), match='2014/11/09 17:00'>, '14:55', '004 疑念'], ['005 使徒', <re.Match object; span=(1, 17), match='2014/11/09 18:00'>, '15:47', '005 使徒'], ['006 奇襲', <re.Match object; span=(1, 17), match='2014/11/09 19:00'>, '15:53', '006 奇襲'], ['007 邸宅', <re.Match object; span=(1, 17), match='2014/11/09 20:00'>, '15:56', '007 邸宅'], ['008 決戦', <re.Match object; span=(1, 17), match='2014/11/09 21:00'>, '15:59', '008 決戦'], ['009 覚悟', <re.Match object; span=(1, 17), match='2014/11/09 22:00'>, '16:05', '009 覚悟'], ['010 港町 ★', <re.Match object; span=(1, 17), match='2014/11/11 19:00'>, '16:09', '010 港町 ★']

In [16]:
# 印出小說標題、小說章節標題以及連結
# 小說標題

print(novel_title, end='\n\n')

# while迴圈加入印出數量限制作法
chapter_title = soup.select('div.index_box dd.subtitle a')
i=0
while i < 50:
    print(chapter_title[i].get_text(), end='')
    print(f" https://ncode.syosetu.com{chapter_title[i]['href']}")
    print('=' * 20)
    i+=1


# # for 迴圈印出全部章節標題+連結做法
# for chapter_title in soup.select('div.index_box dd.subtitle a'):
#     print(chapter_title.get_text(), end='')
#     print(f" https://ncode.syosetu.com{chapter_title['href']}")
#     print('=' * 20)

# # 印出小說連結一覽
i = 0
for i in range(50):
    print(f"{i+1}. {chapter_link[i]}")

美女と賢者と魔人の剣

000 落下 https://ncode.syosetu.com/n4191cj/1/
001 世界 https://ncode.syosetu.com/n4191cj/2/
002 決断 https://ncode.syosetu.com/n4191cj/3/
003 教会 ★ https://ncode.syosetu.com/n4191cj/4/
004 疑念 https://ncode.syosetu.com/n4191cj/5/
005 使徒 https://ncode.syosetu.com/n4191cj/6/
006 奇襲 https://ncode.syosetu.com/n4191cj/7/
007 邸宅 https://ncode.syosetu.com/n4191cj/8/
008 決戦 https://ncode.syosetu.com/n4191cj/9/
009 覚悟 https://ncode.syosetu.com/n4191cj/10/
010 港町 ★ https://ncode.syosetu.com/n4191cj/11/
011 赤毛 https://ncode.syosetu.com/n4191cj/12/
012 迷宮 ★ https://ncode.syosetu.com/n4191cj/13/
013 勧誘 https://ncode.syosetu.com/n4191cj/14/
014 戦術 https://ncode.syosetu.com/n4191cj/15/
015 石像 https://ncode.syosetu.com/n4191cj/16/
016 賢者 https://ncode.syosetu.com/n4191cj/17/
017 捕縛 https://ncode.syosetu.com/n4191cj/18/
018 時間 https://ncode.syosetu.com/n4191cj/19/
019 仲間 https://ncode.syosetu.com/n4191cj/20/
020 大鬼 https://ncode.syosetu.com/n4191cj/21/
021 雷斧 https://ncode.syosetu.com/n4191cj/22/
0

In [17]:
# 抓出各個章節的詳細資訊

# 從上到下依序為章節資訊、章節標題、首次上傳時間、更新時間、連結 (抓出第一筆測試)
chapter_info = soup.select('div.index_box dl.novel_sublist2')
chapter_title = chapter_info[0].a.get_text()
upload_date = chapter_info[0].dt.get_text()
update_date = chapter_info[0].dt.span['title']
novel_link = chapter_info[0].a['href']

# # print(chapter_info)
# print(chapter_title)
# print(upload_date)
# print(update_date)
# print(novel_link)

# 正規表達式 - 從抓到的資料中只抓出日期，另把日期以及時間個別群組化方便用index取用
regex_date = r'([\d]{4}/[\d]+/[\d]+) ([\d]{2}:[\d]{2})'

# 印出章節名、上傳日、更新日(如果有)
for info in chapter_info:
    print(info.a.get_text())
    print(f"アップ日:{re.search(regex_date, info.dt.get_text())[1]}", end='　')
    if info.find('span') != None:
        print(f"改稿日:{re.search(regex_date, info.span['title'])[1]}")
    print(f" https://ncode.syosetu.com{info.a['href']}")

000 落下
アップ日:2014/11/09　改稿日:2019/08/21
 https://ncode.syosetu.com/n4191cj/1/
001 世界
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/2/
002 決断
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/3/
003 教会 ★
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/4/
004 疑念
アップ日:2014/11/09　改稿日:2016/02/15
 https://ncode.syosetu.com/n4191cj/5/
005 使徒
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/6/
006 奇襲
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/7/
007 邸宅
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/8/
008 決戦
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/9/
009 覚悟
アップ日:2014/11/09　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/10/
010 港町 ★
アップ日:2014/11/11　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/11/
011 赤毛
アップ日:2014/11/12　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/12/
012 迷宮 ★
アップ日:2014/11/13　改稿日:2016/01/04
 https://ncode.syosetu.com/n4191cj/13/
01

In [18]:
# 抓取上傳時間、更新時間(更新時間為null時不印)
for info in chapter_info:
    print(info.dt.get_text(), end='')
    if info.find('span') != None:
        print(info.dt.span['title'])
        
# for info in soup.select('div.index_box dl.novel_sublist2'):
#     print(info.dt['long_update'].span['title'])


2014/11/09 17:00（改）2019/08/21 23:58 改稿

2014/11/09 17:00（改）2016/01/04 15:52 改稿

2014/11/09 17:00（改）2016/01/04 15:31 改稿

2014/11/09 17:00（改）2016/01/04 15:53 改稿

2014/11/09 17:00（改）2016/02/15 14:55 改稿

2014/11/09 18:00（改）2016/01/04 15:47 改稿

2014/11/09 19:00（改）2016/01/04 15:53 改稿

2014/11/09 20:00（改）2016/01/04 15:56 改稿

2014/11/09 21:00（改）2016/01/04 15:59 改稿

2014/11/09 22:00（改）2016/01/04 16:05 改稿

2014/11/11 19:00（改）2016/01/04 16:09 改稿

2014/11/12 19:00（改）2016/01/04 16:13 改稿

2014/11/13 19:00（改）2016/01/04 16:17 改稿

2014/11/15 19:00（改）2016/01/04 16:28 改稿

2014/11/16 19:00（改）2016/01/04 16:31 改稿

2014/11/17 19:00（改）2016/01/04 16:37 改稿

2014/11/18 19:00（改）2019/11/15 09:15 改稿

2014/11/19 19:00（改）2016/01/04 16:44 改稿

2014/11/20 19:00（改）2016/01/04 16:47 改稿

2014/11/21 19:00（改）2019/11/15 09:14 改稿

2014/11/22 19:00（改）2019/11/15 09:14 改稿

2014/11/23 19:00（改）2016/01/04 17:14 改稿

2014/11/24 19:00（改）2016/01/04 17:18 改稿

2014/11/25 22:00（改）2016/03/03 21:18 改稿

2014/11/26 19:00（改）2020/08/09 16:48 改稿


In [19]:
# 抓取小說章節標題
section_title = soup.select('div.chapter_title')

for section in section_title:
    print(section.get_text())

プロローグ
第一部　カリス篇
第二部　アシュベル篇
第三部　アンセル篇
第四部　深淵の迷宮篇
第五部　サリータ篇
第六部　絶界の山脈篇
第七部　使徒と魔人篇
第八部　魔人の剣篇
Fragmentary Episode I 『新しい朝へ』
Fragmentary Episode II 『魔人の来訪』
Fragmentary Episode III 『ショートストーリーズ』
設定資料集　（※イラストあり）


In [20]:
# # 建立存放小說的資料夾
# 小說的名字、去除空格後的多餘字元
folderPath = novel_title.split('　')[0] 
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
# 從chapter_link從中抓出一篇小說

# 呼叫chapter_link第一個連結，傳出請求、建立soup物件
res_chapter = req.get(chapter_link[0], headers = my_headers)
soup_chapter = bs(res_chapter.text, 'lxml')

# 取出連結內章節的標題
chapter_title = soup_chapter.select('p.novel_subtitle')[0].get_text()
print(chapter_title, end='\n')

# 無法用於或不建議用於檔案名的非法字元
illegal_c = '[@_!#$%^&*()<>?/|}{~:]　'

# 若檔名包含非法字元，用replace移除
for char in chapter_title:
    if char in illegal_c:
        chapter_title = chapter_title.replace(char, '')

# 取出連結內章節的內文
chapter_content = soup_chapter.select('div#novel_honbun p')
for text in chapter_content:
    print(text.get_text())

In [ ]:
# # 把小說的文字存入檔案，加入換行字元改善可讀性
# with open(f'{folderPath}/{chapter_title}.txt', 'w', encoding='utf-8') as novel_file:
#     novel_file.write(f"{chapter_title}\n\n")
#     for text in chapter_content:
#         novel_file.write(f"{text.get_text()}\n")

In [ ]:
# 把小說標題切到其中不合法字元前一個index，作為儲存檔案資料夾的名字

illegal_c = '[@_!#$%^&*()<>?/|}{~:]　'
folderPath = ''

for char in novel_title:
    if char in illegal_c:
        truncateIndex = novel_title.find(char)
        folderPath = novel_title[0:truncateIndex]
        break
print(folderPath)

### 程式碼改善目標
1. 使可以判讀是否已有已抓到的章節，有時則跳過該章節
2. 設定能智慧截短小說標題避免觸發windows路徑長度限制 (在最短、合理的地方截斷標題)
3. 抓到小說更新時間放在小說內
4. 為各個章節建立子資料夾、加上編號

全角日文字比對用Regex：https://bas-man.dev/post/python/regex-double-byte-python3/

比對不在ASCII編碼中的字元：https://stackoverflow.com/questions/150033/regular-expression-to-match-non-ascii-characters

參考 - 找文字方法：https://www.delftstack.com/howto/python/position-of-character-in-string/

在bs物件內找有沒有子標籤：https://stackoverflow.com/questions/33238091/test-if-children-tag-exists-in-beautifulsoup

==================================
### 找檔案內容用腳本

In [12]:
import os

def search_files(pattern, subdirectory):
    matches = []
    for root, dirs, files in os.walk(os.path.join(os.getcwd(), subdirectory)):
        for file in files:
            with open(os.path.join(root, file), encoding='utf-8') as f:
                if file.endswith('.txt') and pattern in f.read():
                    matches.append(os.path.join(root, file))
    return matches

In [13]:
pattern = input("Enter the string pattern to search for: ")
subdirectory = input("Enter the subdirectory to search in: ")

print(f"The search directory is {os.path.join(os.getcwd(), subdirectory)}")
print(f"The string pattern to search for is: {pattern}")

The search directory is c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣
The string pattern to search for is: レーネ


In [14]:
files = search_files(pattern, subdirectory)
print("The following files contain the matching file names in order:")
for file in files:
    print(file)

The following files contain the matching file names in order:
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\039 幻影.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\040 深層.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\041 派閥.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\042 修練.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\043 侵蝕.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\044 魅了.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\04 第四部　深淵の迷宮篇\045 楔.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\05 第五部　サリータ篇\046 西へ ★.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\05 第五部　サリータ篇\048 機会.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\05 第五部　サリータ篇\049 得意技 ★.txt
c:\Users\John1\GitHub\web_crawler\ncode_novel\美女と賢者と魔人の剣\05 第五部　サリータ篇\050 試練.txt
c:\Users\John1\GitHub\web_crawler\nc